In [5]:
metro_areas = [
    ('Tokyo', 'Japan', 36.933, (35.66667722, 139.6505789)),
    ('Delhi NCR', 'India', 26.454, (28.6139391, 77.2090212)),
    ('Mexico City', 'Mexico', 21.782, (19.4326077, -99.1332087)),
    ('New York-Newark', 'United States', 21.186, (40.7127281, -74.0060152)),
    ('Sao Paulo', 'Brazil', 20.186, (-23.5505199, -46.6333094)),
    ('Shanghai', 'China', 19.200, (31.230416, 121.473701)),
    ('Cairo', 'Egypt', 18.772, (30.0444196, 31.2357116)),
    ('Dhaka', 'Bangladesh', 18.237, (23.8103, 90.4125)),
    ('Mumbai', 'India', 18.042, (19.0760, 72.8777)),
    ('Beijing', 'China', 17.180, (39.9042, 116.4074)),
]

print(f'{"":15} | {"lattitube": >9} | {"longitude": >9}')
for record in metro_areas:
    match record:
        case [name, *_, (lat, lon)] if lon <= 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')
        case [name, _, _, (lat, lon)] if lon > 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f} *')

                | lattitube | longitude
Tokyo           |   35.6667 |  139.6506 *
Delhi NCR       |   28.6139 |   77.2090 *
Mexico City     |   19.4326 |  -99.1332
New York-Newark |   40.7127 |  -74.0060
Sao Paulo       |  -23.5505 |  -46.6333
Shanghai        |   31.2304 |  121.4737 *
Cairo           |   30.0444 |   31.2357 *
Dhaka           |   23.8103 |   90.4125 *
Mumbai          |   19.0760 |   72.8777 *
Beijing         |   39.9042 |  116.4074 *


In [3]:
t = (1, 2, [34, 34])
t[2] += [1, 2]

TypeError: 'tuple' object does not support item assignment

In [11]:
import array
a = array.array('l', [-11111111, 22222222, -33333333, 44444444])
m = memoryview(a)

In [9]:
def sample_generator():
    yield 1
    print('Inside generator')
    yield 2
    print('Still inside generator')
    yield 3
    print('Exiting generator')

g = sample_generator()
print(next(g))
print(next(g))
print(next(g))
print(next(g))

1
Inside generator
2
Still inside generator
3
Exiting generator


StopIteration: 

In [ ]:
import random
import string

def generate_isin(country_code='US'):
    """Generate a custom ISIN (International Securities Identification Number)."""
    random_part = ''.join(random.choices(string.ascii_uppercase + string.digits, k=9))
    checksum = str(sum(int(c) for c in random_part if c.isdigit()) % 10)
    return f"{country_code}{random_part}{checksum}"

# Example usage
custom_isin = generate_isin()
print(f"Generated ISIN: {custom_isin}")

In [ ]:
from datetime import datetime, timedelta
from dateutil import rrule
from collections.abc import Iterable

class ISIN:
    def __init__(self, code: str):
        if not self._validate_isin(code):
            raise ValueError(f"Invalid ISIN: {code}")
        self.code = code

    @staticmethod
    def _validate_isin(code: str) -> bool:
        """Validate the ISIN format."""
        if len(code) != 12:
            return False
        if not code[:2].isalpha() or not code[2:11].isalnum() or not code[-1].isdigit():
            return False
        return True

    def __repr__(self):
        return f"ISIN('{self.code}')"

    def __str__(self):
        return self.code


class BondPrice:
    __slots__ = ('__bond_yield', '__price', '__coupon_rate', '__expiry_date', '__face_value', '__coupon_dates')

    def __init__(self, bond_yield: float, coupon_rate: float, expiry_date: datetime, face_value: float, semi_annual: bool = True):
        self.__expiry_date = expiry_date  # Store expiry_date as a datetime object
        self.__bond_yield = bond_yield / 2 if semi_annual else bond_yield
        self.__coupon_rate = coupon_rate / 2 if semi_annual else coupon_rate
        self.__face_value = face_value
        self.__coupon_dates = []  # Initialize coupon dates
        self.__set_coupon_dates(semi_annual=semi_annual)
        self.calculate_price()

    def calculate_price(self, dirty=False):
        """Calculate the price of the bond based on yield, coupon rate, and expiry date."""
        time_to_maturity = (self.__expiry_date - datetime.now()).days / 365.0
        self.__price = self.__face_value / (1 + self.__bond_yield) ** time_to_maturity + self.__calculate_coupon_price()
        if dirty:
            self.__price += self.__calculate_accrued_coupon()

    def get_price(self):
        """Get the calculated price of the bond."""
        return self.__price

    def get_coupon_dates(self):
        """Get the coupon dates for the bond."""
        return self.__coupon_dates

    def __set_coupon_dates(self, semi_annual=True):
        """Set the coupon dates for the bond."""
        payment_months = (1, 7) if semi_annual else (1,)
        self.__coupon_dates = [
            datetime(year=date.year, month=date.month, day=min(self.__expiry_date.day, 28))
            for date in rrule.rrule(rrule.MONTHLY, dtstart=datetime.now(), until=self.__expiry_date)
            if date.month in payment_months
        ]

    @staticmethod
    def __get_last_coupon_date(semi_annual=True):
        """Get the last coupon date based on the current date."""
        dates = list(rrule.rrule(rrule.MONTHLY, dtstart=datetime.now() - timedelta(days=365), until=datetime.now()))
        for i in reversed(dates):
            if semi_annual:
                if i.month in (1, 7):  # Check for January or July
                    return i
            else:
                if i.month == 1:  # Check for January
                    return i
        return None  # Return None if no matching date is found

    def __calculate_accrued_coupon(self, semi_annual=True):
        """Calculate the accrued coupon based on the coupon rate and time to maturity."""
        last_coupon_date = self.__get_last_coupon_date(semi_annual)
        if not last_coupon_date:
            return 0  # No accrued coupon if no last coupon date is found
        days_since_last_coupon = (datetime.now() - last_coupon_date).days
        return (self.__coupon_rate * self.__face_value) * (days_since_last_coupon / 365)

    def __calculate_coupon_price(self):
        """Calculate the coupon price based on the coupon rate and time to maturity."""
        coupon_price = 0
        for coupon_date in self.__coupon_dates:
            time_to_coupon = (coupon_date - datetime.now()).days / 365.0
            coupon_price += self.__coupon_rate * self.__face_value / (1 + self.__bond_yield) ** time_to_coupon
        return coupon_price

    def __repr__(self):
        return f'Price: {self.__price}'


class Bond:
    def __init__(self, isin: ISIN, issuer: str, bond_yield: float, coupon_rate: float, face_value: float):
        self.isin = isin
        self.name = issuer
        self.bond_yield = bond_yield
        self.coupon_rate = coupon_rate
        self.face_value = face_value

    def __repr__(self):
        return f'Bond({self.name}, {self.bond_yield}, {self.coupon_rate})'

    def __str__(self):
        return f'{self.name} - {self.bond_yield} - {self.coupon_rate}'


class BondPortfolio:
    def __init__(self, bonds: Iterable[Bond]):
        if not isinstance(bonds, Iterable):
            raise TypeError("Bonds must be an iterable.")
        self.bonds = bonds

    def __iter__(self):
        yield from self.bonds

    def __len__(self):
        return len(self.bonds)

    def __getitem__(self, isin):
        for bond in self.bonds:
            if bond.isin == isin:
                return bond
        raise KeyError(f"Bond with ISIN {isin} not found.")

    def __contains__(self, bond):
        return bond in self.bonds

    def __repr__(self):
        return ','.join([str(bond.isin) for bond in self.bonds])


# Example Usage
bond_price = BondPrice(
    bond_yield=0.075,
    coupon_rate=0.05,
    expiry_date=datetime.today() + timedelta(days=365 * 20),
    face_value=1000
)
print(f"Bond Price: {bond_price.get_price()}")
print(f"Coupon Dates: {bond_price.get_coupon_dates()}")

AttributeError: 'BondPrice' object has no attribute 'get_price'

In [1]:
class Strange:
    def __init__(self, val):
        self.val = val

    def __add__(self, other):
        return self.val * other.val

    def __radd__(self, other):
        return self.val + other

a = Strange(3)
b = Strange(2)

print(a + b)
print(10 + a)
print(a + 10)

6
13


AttributeError: 'int' object has no attribute 'val'

In [2]:
class Organism:
    def describe(self):
        return "Basic lifeform"

    class Cell:
        def function(self):
            return "Generic cell"

class Animal(Organism):
    def describe(self):
        return "I am an animal"

    class Cell(Organism.Cell):
        def function(self):
            return "Animal cell with specialized roles"

print(Animal().describe(), Animal.Cell().function())

I am an animal Animal cell with specialized roles


In [5]:
from itertools import zip_longest

arr = [1] * 2
arr2 = [1, 3] * 2
arr3 = [1] * 3
ans = 0

for i, j, k in zip_longest(arr, arr2, arr3, fillvalue=2):
    print((i + j), k)
    ans += (i + j) * k

print(ans)

2 1
4 1
3 1
5 2
19


In [6]:
class A():
    __num = 5

    def getNum(self):
        return self.__num

a = A()
a.__num = 4

print(f"{a.getNum()}{len(a.__dict__)}{a.__num}")

514


In [8]:
class A():
    def normalMethod(self):
        print(1, end = '')

class B(A):
    def normalMethod(self):
        print(2, end = '')
        return super().normalMethod()

class C(A):
    def normalMethod(self):
        print(3, end = '')
    
class D(B, C):
    def normalMethod(self):
        print(4, end = '')
        super().normalMethod()

d = D()
d.normalMethod()

423

In [10]:
class MyClass():
    def __init__(self, num):
        self.mine = num
    
    def __eq__(self, other):
        if isinstance(other, MyClass):
            return self.mine - other.mine
        
class YourClass():

    def __init__(self, num):
        self.yours = num

    def __eq__(self, other):
        if isinstance(other, YourClass):
            return self.yours + other.yours
        elif isinstance(other, MyClass):
            return self.yours * other.mine
        else:
            return 3

print(MyClass(4) == YourClass(2))

None


In [11]:
text = "Python"[::-1][::-1]
print(text)

Python


In [3]:
import contextlib

class Origin:
    def __init__(self):
        print("The origin")

class A(Origin):
    def __init__(self):
        print("A's init")
        super().__init__()

class B(Origin):
    def __init__(self):
        print("B's init")
        super().__init__()

class TestContext(A, B):
    def __enter__(self):
        return 'Has Entered and been initialized'
    
    def __exit__(self, exec_type, exec_value, traceback):
        print("---------")
        print(exec_type)
        print(exec_value)
        print(traceback)

@contextlib.contextmanager
def testcontext():
    message = 'origin'

    try:
        yield 'Has Entered and been initialized'
        print("SUCCC")
        yield 'Here'
    except Exception as e:
        print(e)
    finally:
        print(message)


# with testcontext() as te:
#     print(te + 1)

# with TestContext() as text:
#     print(text + 1)

@testcontext()
def test():
    print("Testing things")

In [4]:
test()

Testing things
SUCCC
origin


RuntimeError: generator didn't stop

In [ ]:
from typing import ChainMap, Any
class Environment(ChainMap[int, Any]):

    def change(self, key: int, value: Any):
        for map in self.maps:
            if key in map:
                map[key] = value
                return
    
        raise KeyError
    
inner_env = { 1: 'something ', 2: 'else'}
outer_env = { 3: 'or is it ', 4: 'this'}

env = Environment(inner_env, outer_env)
env[3] = 'new stuff'
env

Environment({1: 'something ', 2: 'else', 3: 'new stuff'}, {3: 'or is it ', 4: 'this'})

In [12]:
id(env)

2605098037136

In [15]:
from math import fma

print(int(fma(2, 2, 1_10_0)))

1104


In [17]:
arr1 = [1, 2]
arr2 = [1, 2, 3, 4]
arr3 = list(zip(arr1,arr2))
print(arr3)

[(1, 1), (2, 2)]


In [4]:
a = 1
b = 1
if (a is b):
    print("A")
else:
    print("B")

a = 257
b = 257
if (a is b):
    print("C")
else:
    print("D")

a = int("-6")
b = -6
if (a is b):
    print("E")
else:
    print("F")

A
D
F


In [6]:
t = (1, 2, [30, 40])
t[2] += [50, 60]

TypeError: 'tuple' object does not support item assignment

In [7]:
t

(1, 2, [30, 40, 50, 60])

In [ ]:
a = 257
b = 257
if (a is b):
    print("C")
else:
    print("D")

: 

In [2]:
A = [1,2,3,4,5,6,612434,2342343,34,34234]
B = A.copy()
A.sort()

In [3]:
B

[1, 2, 3, 4, 5, 6, 612434, 2342343, 34, 34234]

In [4]:
A

[1, 2, 3, 4, 5, 6, 34, 34234, 612434, 2342343]

In [5]:
sorted(B)

[1, 2, 3, 4, 5, 6, 34, 34234, 612434, 2342343]

In [8]:
class A():
    __num = 5

    def getNum(self):
        return self.__num

a = A()
print(a.__dict__)
a.__num = 4
a.__dict__

{}


{'__num': 4}

In [1]:
import os
os.cpu_count()

16

In [ ]:
import httpx

client = httpx.AsyncClient()
task = client.post('http://localhost:8080')
response = await task
print(response)

In [15]:
from datetime import datetime

datetime.now().__class__

datetime.datetime

In [23]:
def config(default=None, **kwargs, ):
    kwargs['default'] = kwargs.get('default', default)
    return kwargs

result = config(setting=1, default=2)
print(result['default'])

2


In [31]:
class A(type):

    def __init__(c, *args, **kwargs):
        print("1", end = "")
        super().__init__(*args, **kwargs)

    def __call__(c, *args, **kwargs):
        print("2", end = "")
        type.__call__(c, *args, **kwargs)

    def __new__(c, *args, **kwargs):
        print("3", end = "")
        return super().__new__(c, *args, **kwargs)

class B:

    def __init__(s):
        print("4", end = "")

_ = B()
_ = B()

44

In [ ]:
class A:
    def __init__(self, value):
        self.val = value
    def __hash__(self):
        return hash(self.val)

class B:
    def __init__(self, value):
        self.val = value

    def __eq__(self, other):
        return isinstance(other, B) and self.val == other.val
    
    def __hash__(self):
        return hash(self.val)

dict = dict()

a = True
b = 1
dict[a] = 1
dict[b] = 2

c = A(3)
d = A(3)
dict[c] = 3
dict[d] = 4

e = B(4)
f = B(4)
dict[e] = 5
dict[f] = 6

print(dict.values())

dict_values([2, 3, 4, 6])


In [5]:
dict

{True: 2,
 <__main__.A at 0x20bb9852120>: 3,
 <__main__.A at 0x20bb979c050>: 4,
 <__main__.B at 0x20bb9851fd0>: 6}

: 